In [4]:
import numpy as np
import pandas as pd
import os
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from vader_feature import new_words

In [6]:
def remove_pattern(df, patt, column_name):
    patt_find = df[column_name].str.findall(patt, flags = re.IGNORECASE)
    valid_index = np.where(patt_find.apply(len) == 0, True, False)
    return valid_index

In [7]:
url_patt = "(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$"
word_patt  = r'\bfree\b|\binstall\b|\bdownload\b|\bapp\b|\bwallet\b|\bhardware\b|\blink\b|\bcard\b|\bpayapl\b|\bmining\b|\bmine\b|\bplatform\b|\bbrowser\b|\bsite\b'

In [8]:
def get_vader_df(df):
    title_vader = df['title'].dropna().apply(analyzer.polarity_scores)
    #df로 변환해 반환해준다.
    vader_title_df = pd.DataFrame(dict(title_vader)).T
    return vader_title_df #vader실행 결과를 반환해준다.

In [ ]:
column_name = 'title'
path = '../dataset/'
analyzer = SentimentIntensityAnalyzer() #vader 객체
analyzer.lexicon.update(new_words)
for f in os.listdir(path):
    df = pd.read_csv(f'{path}{f}')
    df = df.drop_duplicates(['title'],keep='first') #중복 처리를 해준다.
    df = df.drop('selftext',axis = 1)
    word_index = remove_pattern(df, word_patt, column_name)
    url_index = remove_pattern(df,url_patt,column_name)
    title_to_remove = df.loc[(word_index & url_index) == False, ['title']]
    title_valid = df.loc[~df.index.isin(title_to_remove.index)]
    title_valid['title_vader'] = get_vader_df(title_valid)['compound']
    title_valid = title_valid[['id','author','title_vader','title','created_utc','num_comments','score']]
    title_valid.to_csv(f'./valid_data/{f}.csv')
    # title_to_remove.to_csv(f'./remove_data/{f}.csv')

In [9]:
path = './valid_data/'
file_list = os.listdir(path)

In [19]:
def create_train_data(df):
    indexes = np.abs(df['title_vader']) > 0.6
    return indexes

In [20]:
for f in file_list:
    df = pd.read_csv(path+f)
    df = df.loc[create_train_data(df)]
    if os.path.exists('./train_data.csv'):
            df.to_csv('./train_data.csv',sep=',',mode='a',index=False,header=False)
    else:
        df.to_csv("./train_data",sep=',',index=False)

In [1]:
df = pd.read_csv("./train_data")

df = df.drop('Unnamed: 0',axis=1)

NameError: name 'pd' is not defined

In [23]:
df.to_csv("./train_data",sep=',')